In [100]:
from dkibo import DKIBO, UtilityFunction
import pandas as pd
import numpy as np
import warnings
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import Module
from scipy.stats import norm
import copy
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from utils import round_result


previous_experiment = pd.read_excel(r'C:\Users\darkn\Desktop\data\POD 1~150.xlsx')

target = previous_experiment.columns[1]
a = previous_experiment[target].apply(lambda x: x.split('   ')).to_list()
for i in range(len(a)):
    for j in range(len(a[i])):
        a[i][j] = eval(a[i][j])
a = np.array(a)
b = previous_experiment[previous_experiment.columns[0]].to_numpy()

small_indices = np.argpartition(b, 50)[:50]
mask = np.ones(a.shape[0], dtype=bool)
mask[small_indices] = False
a = a[mask, :]
b = np.delete(b, small_indices)


previous_experiment = pd.read_excel(r'C:\Users\darkn\Desktop\data\with_structure\BO_new(3)1.xlsx')
b1 = previous_experiment['KM/mM'].to_numpy()
a1 = previous_experiment[['Mn', 'Fe', 'V', 'Cu', 'Co']]
a1 = np.array(a1)
a = np.concatenate([a, a1])
b = np.concatenate([b, b1])


class Model(Module):
    def __init__(self):
        super(Model, self).__init__()
        self.Layer1 = nn.Linear(in_features=5, out_features=256)
        self.Layer2 = nn.Linear(in_features=256, out_features=128)
        self.Layer3 = nn.Linear(in_features=128, out_features=8)

    def forward(self, x):
        x = F.relu(self.Layer1(x))
        x = F.relu(self.Layer2(x))
        x = self.Layer3(x)

        return x


structure_model = torch.load('POD-ml-premodel.pkl')
structure_model.eval()


class NNWrapper:
    def __init__(self, nn_model: Module):
        self._input_model = nn_model
        self._output_model = RandomForestRegressor()

    def fit(self, X, y):
        """Fit internal neural network model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features) or list of object
            Feature vectors or other representations of training data.

        y : array-like of shape (n_samples,) or (n_samples, n_targets)
            Target values

        Returns
        -------
        self : returns an instance of self.
        """
        x_tensor = torch.Tensor(X)
        x_structure = self._input_model(x_tensor).detach().numpy()
        self._output_model.fit(x_structure, y)

    def predict(self, X):
        x_tensor = torch.Tensor(X)
        x_structure = self._input_model(x_tensor).detach().numpy()
        return self._output_model.predict(x_structure)


nn_wrapper = NNWrapper(structure_model)

bound = {
    'Mn': (5, 35),
    'Fe': (5, 35),
    'V': (5, 35),
    'Cu': (5, 35),
    'Co': (5, 35),
}


bo = DKIBO(f=None,
           pbounds=bound,
           ml_regressor=nn_wrapper)


for i in range(len(a)):
    # bo.register(line[1:], line[0])
    bo.register(a[i], -b[i])
    
constraint = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] + x[2] + x[3] + x[4] - 100})

BATCH = 8
RESAMPLE = 1

all_results = []
x_results = []
i = 0

x_init = copy.deepcopy(bo._space.params[:5])

In [101]:
x_init

array([[10., 20., 10., 30., 30.],
       [10., 20., 20., 20., 30.],
       [10., 20., 20., 30., 20.],
       [10., 20., 30., 20., 20.],
       [10., 20., 30., 30., 10.]])

In [102]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    while i < BATCH:
        k = np.random.uniform(0, 10)
        util = UtilityFunction(kind='ucb',
                               # kappa=2.576 + noise,
                               kappa=k,
                               xi=0.0,
                               kappa_decay=1,
                               kappa_decay_delay=0,
                               ml_regressor=bo.ml_regressor,
                               x_init=x_init,
                               max_iter=100)
        result_list = []
        x_probe_list = []
        for _ in range(RESAMPLE):
            x_probe = bo.suggest(util, constraint)
            x_probe_list.append(x_probe)
            result_list.append(
                util.utility(np.array(list(x_probe.values())).reshape(1, -1), bo._gp, bo._space.target.max())[0])
        max_index = np.argmax(result_list)
        x_probe = x_probe_list[max_index]
        x = np.array(list(x_probe.values()))
        mean, std = bo._gp.predict(x.reshape(1, -1), return_std=True)

        if round_result(x_probe, constraint) in x_results:
            continue
        else:
            print(round_result(x_probe, constraint))
            x = np.array(list(x_probe.values()))
            mean, std = bo._gp.predict(x.reshape(1, -1), return_std=True)
            print(mean, std)

            # bo.register(x, mean[0])

            x_results.append(round_result(x_probe, constraint))
            all_results.append((round_result(x_probe, constraint), mean, std))
            i += 1
print(all_results)

{'Co': 11.0, 'Cu': 14.0, 'Fe': 29.0, 'Mn': 18.0, 'V': 28.0}
[-50.56074277] [23.01343353]
{'Co': 21.0, 'Cu': 20.0, 'Fe': 21.0, 'Mn': 13.0, 'V': 25.0}
[-36.46712433] [19.16339636]
{'Co': 10.0, 'Cu': 16.0, 'Fe': 28.0, 'Mn': 21.0, 'V': 25.0}
[-30.95955813] [19.73598428]
{'Co': 7.0, 'Cu': 15.0, 'Fe': 33.0, 'Mn': 21.0, 'V': 24.0}
[-46.4370787] [22.93007165]
{'Co': 11.0, 'Cu': 15.0, 'Fe': 27.0, 'Mn': 20.0, 'V': 27.0}
[-42.85524957] [22.4649486]
{'Co': 16.0, 'Cu': 16.0, 'Fe': 24.0, 'Mn': 21.0, 'V': 23.0}
[-50.80923711] [23.01407329]
{'Co': 13.0, 'Cu': 15.0, 'Fe': 27.0, 'Mn': 19.0, 'V': 26.0}
[-47.04913168] [22.91457903]
{'Co': 5.0, 'Cu': 17.0, 'Fe': 27.0, 'Mn': 21.0, 'V': 30.0}
[-50.04382007] [23.00974001]
[({'Co': 11.0, 'Cu': 14.0, 'Fe': 29.0, 'Mn': 18.0, 'V': 28.0}, array([-50.56074277]), array([23.01343353])), ({'Co': 21.0, 'Cu': 20.0, 'Fe': 21.0, 'Mn': 13.0, 'V': 25.0}, array([-36.46712433]), array([19.16339636])), ({'Co': 10.0, 'Cu': 16.0, 'Fe': 28.0, 'Mn': 21.0, 'V': 25.0}, array([-30.95

In [96]:
xs = []

for pair in all_results:
    xs.append(list(pair[0].values()))
    
print(bo.ml_regressor.predict(xs))
print(bo._gp.predict(xs, return_std=True))

[-37.86045 -33.44865 -31.27056 -30.95902 -34.28884 -35.42785 -33.30537
 -32.45078]
(array([-52.71523182, -52.71523182, -52.71523182, -52.71523182,
       -52.71523182, -52.7152195 , -52.71523182, -52.71523182]), array([23.10175786, 23.10175786, 23.10175786, 23.10175786, 23.10175786,
       23.10175786, 23.10175786, 23.10175786]))


In [99]:
util._regression_scale_rate

1

In [55]:
gp = GaussianProcessRegressor(
            kernel=Matern(nu=2.5, length_scale=1.0),
            alpha=1e-6,
            normalize_y=True,
            n_restarts_optimizer=5,
)
gp.fit(bo._space.params, bo._space.target)

C:\Users\darkn\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


GaussianProcessRegressor(alpha=1e-06, kernel=Matern(length_scale=1, nu=2.5),
                         n_restarts_optimizer=5, normalize_y=True)

In [57]:
gp.predict([[0, 3, 20, 5, 100]], return_std=True)

(array([-53.82508548]), array([22.95410459]))

In [58]:
previous_experiment = pd.read_excel(r'C:\Users\darkn\Desktop\data\POD 1~150.xlsx')

target = previous_experiment.columns[1]
a = previous_experiment[target].apply(lambda x: x.split('   ')).to_list()
for i in range(len(a)):
    for j in range(len(a[i])):
        a[i][j] = eval(a[i][j])
a = np.array(a)
b = previous_experiment[previous_experiment.columns[0]].to_numpy()

bound = {
    'Mn': (5, 35),
    'Fe': (5, 35),
    'V': (5, 35),
    'Cu': (5, 35),
    'Co': (5, 35),
}


bo = DKIBO(f=None,
           pbounds=bound,
           ml_regressor=None)

for i in range(len(previous_experiment)):
    line = previous_experiment.iloc[i].to_numpy()
    # bo.register(line[1:], line[0])
    bo.register(a[i], -b[i])

previous_experiment = pd.read_excel(r'C:\Users\darkn\Desktop\data\BO.xlsx')
b = previous_experiment['KM/mM'].to_numpy()
a = previous_experiment[['Mn', 'Fe', 'V', 'Cu', 'Co']]

for i in range(len(previous_experiment)):
    line = a.iloc[i].to_numpy()
    bo.register(line, -b[i])
    
bo._gp = bo._gp.fit(bo._space.params, bo._space.target)

In [87]:
new_bo =   DKIBO(f=None,
           pbounds=bound,
           ml_regressor=None)
for i in range(len(a)):
    # bo.register(line[1:], line[0])
    new_bo.register(a[i], -b[i])

new_bo._gp.fit(new_bo._space.params, new_bo._space.target)

GaussianProcessRegressor(alpha=1e-06,
                         kernel=Matern(length_scale=[1, 1, 1, 1, 1], nu=2.5),
                         n_restarts_optimizer=5, normalize_y=True,
                         random_state=RandomState(MT19937) at 0x2439206D140)

In [89]:
new_bo._gp.predict(xs, return_std=True)

(array([-53.82508548, -53.82508548, -53.82508548, -53.82508548,
        -53.82508548, -53.82508548, -53.82508548, -53.82508548]),
 array([22.95410459, 22.95410459, 22.95410459, 22.95410459, 22.95410459,
        22.95410459, 22.95410459, 22.95410459]))

# Multi-Objective DKIBO Test

In [6]:
from multi_objective_dkibo import MultiObjectiveDKIBO
import numpy as np
import pandas as pd

def target_function(x):
    y0 = np.sum(x) - np.sqrt(x[3]) - np.sqrt(x[2])
    y1 = 5 * np.sin(x[0]) - x[1] ** 2
    y2 = x[0] ** 2 + x[3] ** 2 + np.sqrt(x[2])
    return [y0, y1, y2]

In [3]:
bo = MultiObjectiveDKIBO(
    objective_num=3,
    pbounds={'n0':(0, 1), 'n1':(0, 1), 'n2':(0, 1), 'n3':(0, 1)},
    ml_regressor=[None, None, None]
)

In [7]:
bo._prime_queue(5)
init_points = bo._queue._queue
for point in init_points:
    bo.register(point, target_function(point))

In [8]:
bo.suggest('ucb', batch_size=8)

IndexError: invalid index to scalar variable.

In [4]:
from numpy.random import rand, randn
from scipy.linalg import qr


In [5]:
r, N = 6, 10

# Random orthogonal vectors
eig_vecs, _ = qr(randn(N, r), mode='economic')
# Random eigenvalues
eig_vals = rand(r)  # 0< <1

In [6]:
(eig_vecs*eig_vals).dot(eig_vecs.T)

array([[ 0.33838093,  0.03641226, -0.131701  , -0.10685541,  0.00532905,
        -0.05858034, -0.11296059, -0.09131955, -0.00611408,  0.04442329],
       [ 0.03641226,  0.31580411,  0.13971558,  0.07651349,  0.04799645,
        -0.14440103,  0.05526061,  0.07887735, -0.03756414, -0.12332591],
       [-0.131701  ,  0.13971558,  0.23223303, -0.05611139, -0.0514756 ,
        -0.03426262, -0.0443968 ,  0.10679425,  0.02636799, -0.11265734],
       [-0.10685541,  0.07651349, -0.05611139,  0.42818082, -0.0324862 ,
         0.04216618,  0.16940781, -0.01445149, -0.05295544, -0.03408465],
       [ 0.00532905,  0.04799645, -0.0514756 , -0.0324862 ,  0.27982463,
        -0.13532191,  0.06175365, -0.15320781, -0.04034324, -0.05041777],
       [-0.05858034, -0.14440103, -0.03426262,  0.04216618, -0.13532191,
         0.33458864,  0.1234255 ,  0.06384687, -0.17200815, -0.17133807],
       [-0.11296059,  0.05526061, -0.0443968 ,  0.16940781,  0.06175365,
         0.1234255 ,  0.34229315,  0.15315322

In [1]:
from dppy.finite_dpps import FiniteDPP
from dppy.utils import example_eval_L_linear

In [9]:
DPP = FiniteDPP('correlation', **{'K': (eig_vecs*eig_vals).dot(eig_vecs.T)})

In [11]:
for _ in range(10):
    DPP.sample_exact()

DPP.list_of_samples

[[3, 0, 2, 6, 4],
 [5, 8, 3, 0],
 [1, 3, 0, 5],
 [4, 9, 2],
 [6, 7, 3, 9],
 [6, 1, 7],
 [1, 8, 3, 0],
 [6, 0, 9],
 [9],
 [7, 0, 5, 9]]

In [9]:
import numpy
from deap import base, creator
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMax)

In [12]:
a = numpy.array([1, 2, 3])
b = numpy.array([1, 2, 3])
operator.eq(a, b)

NameError: name 'operator' is not defined

In [4]:
import numpy as np
from deap import base
from deap import creator
from deap import tools
import random
import array

def uniform(bounds):
    return [random.uniform(b[0], b[1]) for b in bounds]

objective = lambda x: [x[0], x[1], x[2]]
MU = 100
NGEN=100
CXPB=0.9
pbounds = np.array([[0, 1], [0, 1], [0, 1], [0, 1]])
NObj = 3

creator.create("FitnessMin", base.Fitness, weights=(-1.0,)*NObj)
creator.create("Individual", array.array, typecode='d',
               fitness=creator.FitnessMin)
    
toolbox = base.Toolbox()

NDIM = len(pbounds)

toolbox.register("attr_float", uniform, pbounds)

toolbox.register("individual",
                 tools.initIterate,
                 creator.Individual,
                 toolbox.attr_float)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", objective)

toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,
                 low=pbounds[:, 0].tolist(),
                 up=pbounds[:, 1].tolist(),
                 eta=20.0)

toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=pbounds[:, 0].tolist(),
                 up=pbounds[:, 1].tolist(),
                 eta=20.0,
                 indpb=1.0/NDIM)

toolbox.register("select", tools.selNSGA2)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

logbook = tools.Logbook()
logbook.header = "gen", "evals", "std", "min", "avg", "max"

pop = toolbox.population(n=MU)

In [10]:
a = pop[0]
a[0]

0.6106079208445867